# Episode 2
## Questions
How do we transform celestial coordinates from one frame to another and save a subset of the results in files?

## Objectives
Use Python string formatting to compose more complex ADQL queries.
Work with coordinates and other quantities that have units.
Download the results of a query and store them in a file.

## Outline
We’ll start with an example that does a “cone search”; that is, it selects stars that appear in a circular region of the sky.
Then, to select stars in the vicinity of GD-1, we will:
- Use `Quantity` objects to represent measurements with units.
- Use `Astropy` to convert coordinates from one frame to another.
- Use the ADQL keywords `POLYGON`, `CONTAINS`, and `POINT` to select stars that fall within a polygonal region.
- Submit a query and download the results.
- Store the results in a `FITS` file.

In [1]:
## import block
import astropy.units as u
import numpy as np

from os.path import getsize

from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from gala.coordinates import GD1Koposov10

from astro_utils.geometry import make_sky_rectangle
from astro_utils.formatting import skycoord_to_string

In [2]:
# create angle
angle = 10 * u.degree
print(f'{angle}\n{type(angle)}')

10.0 deg
<class 'astropy.units.quantity.Quantity'>


In [3]:
# convert to arcmin
angle_arcmin = angle.to(u.arcmin)
print(angle_arcmin)

600.0 arcmin


In [4]:
angle + 30 * u.arcmin

<Quantity 10.5 deg>

In [5]:
try: 
    angle + 5 * u.kg
except Exception as e:
    print(f'There was an exception.')

There was an exception.


### Exercise 2.1
Create a quantity that represents `5` arcminutes and assign it to a variable called radius.

Then convert it to degrees.

In [6]:
angle = 5 * u.arcmin
angle = angle.to(u.degree)
print(angle)

0.08333333333333333 deg


## Selecting a Region

In [7]:
cone_query1 = """SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1 = CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

cone_job1 = Gaia.launch_job(cone_query1)
cone_results1 = cone_job1.get_results()
cone_results1

SOURCE_ID
int64
3322773965056065536
3322773758899157120
3322774068134271104
3322773930696320512
3322774377374425728
3322773724537891456
3322773724537891328
3322773930696321792
3322773724537890944


### Exercise 2.2
When you are debugging queries like this, you can use `TOP` to limit the size of the results, but then you still don’t know how big the results will be.

An alternative is to use `COUNT`, which asks for the number of rows that would be selected, but it does not return them.

In the previous query, replace `TOP 10 source_id` with `COUNT(source_id)` and run the query again. How many stars has Gaia identified in the cone we searched?

In [8]:
cone_query2 = """SELECT 
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1 = CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

cone_job2 = Gaia.launch_job(cone_query2)
cone_results2 = cone_job2.get_results()
cone_results2

COUNT
int64
594


## Transforming Coordinates

In [9]:
# create point in ICRS
ra = 88.8 * u.degree
dec = 7.4 * u.degree
coord_icrs = SkyCoord(ra=ra, dec=dec, frame='icrs')

coord_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (88.8, 7.4)>

In [10]:
# transform to galactic
coord_galactic = coord_icrs.transform_to('galactic')
coord_galactic

<SkyCoord (Galactic): (l, b) in deg
    (199.79693102, -8.95591653)>

In [11]:
# transform to GD-1, as defined by our reference frame
gd1_frame = GD1Koposov10()
coord_gd1 = coord_icrs.transform_to(gd1_frame)
coord_gd1

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    (-94.97222038, 34.5813813)>

### Exercise 2.3
Find the location of GD-1 in ICRS coordinates.

Create a SkyCoord object at `0°, 0°` in the GD-1 frame.

Transform it to the ICRS frame.

Hint: Because ICRS is a standard frame, it is built into Astropy. You can specify it by name, icrs (as we did with galactic).

In [12]:
gd1_phi1 = 0 * u.degree
gd1_phi2 = 0 * u.degree

coord_gd1 = SkyCoord(phi1=gd1_phi1, phi2=gd1_phi2, frame=GD1Koposov10())
coord_gd1.transform_to('icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (200., 59.4504341)>

## Selecting a rectangle
Now that we know how to define coordinate transformations, we are going to use them to get a list of stars that are in GD-1. As we mentioned before, this is a lot of stars, so we are going to start by defining a rectangle that encompasses a small part of GD-1. This is easiest to define in GD-1 coordinates.

In [13]:
# Define the rectangle in GD1 coords
phi1_min = -55 * u.degree 
phi1_max = -45 * u.degree
phi2_min = -8 * u.degree
phi2_max = 4 * u.degree

corners = make_sky_rectangle(phi1_min, phi1_max, phi2_min, phi2_max, GD1Koposov10())


In [14]:
# Transform to ICRS
corners_icrs = corners.transform_to('icrs')
corners_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254),
     (146.27533314, 19.26190982)]>

## Defining a polygon
In order to use this polygon as part of an ADQL query, we have to convert it to a string with a comma-separated list of coordinates, as in this example.

In [15]:
corners_icrs_string = ' '.join(corners_icrs.to_string()).replace(' ',', ')
corners_icrs_string

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

In [16]:
skycoord_to_string(corners_icrs)

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

## Assembling the Query

In [17]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'

polygon_query_base = """
SELECT 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON({sky_point_list}))
"""

polygon_query = polygon_query_base.format(columns=columns, sky_point_list=skycoord_to_string(corners_icrs))

polygon_query_job = Gaia.launch_job_async(polygon_query)
polygon_results = polygon_query_job.get_results()
print(len(polygon_results))


INFO: Query finished. [astroquery.utils.tap.core]
140339


In [18]:
filename = 'gd1_results.fits'
polygon_results.write(filename, overwrite=True)

getsize(filename) / 1024 / 1024

6.4324951171875

## Summary
In this notebook, we composed more complex queries to select stars within a polygonal region of the sky. Then we downloaded the results and saved them in a FITS file.

In the next notebook, we’ll reload the data from this file and replicate the next step in the analysis, using proper motion to identify stars likely to be in GD-1.

KEY POINTS
- For measurements with units, use Quantity objects that represent units explicitly and check for errors.
- Use the format function to compose queries; it is often faster and less error-prone.
- Develop queries incrementally: start with something simple, test it, and add a little bit at a time.
- Once you have a query working, save the data in a local file. If you shut down the notebook and come back to it later, you can reload the file; you don’t have to run the query again.
